In [ ]:


# --- 1. Setup and Library Imports ---
import numpy as np
import pandas as pd
import math
from dataclasses import dataclass
import matplotlib.pyplot as plt

print('Libraries imported. Ready to run.')


## Problem 1 — Min–Max Normalization & Sigmoid NN Forward Pass  
Prompt excerpt: Normalize inputs (AGE, INC) and compute output of a given 2-layer NN with sigmoid activations for inputs **AGE=70**, **INCOME=50K** 【112†Assignment 4.pdf†L1-L50】.

**Normalization:**  
- AGE range: 20–80 → min=20, max=80  
- INCOME range: 10K–110K → min=10, max=110  

Normalized values:
\[ \text{age\_norm} = \frac{70-20}{80-20} = \frac{50}{60} = 0.8333\overline{3} \]
\[ \text{inc\_norm} = \frac{50-10}{110-10} = \frac{40}{100} = 0.4 \]

The figure in the PDF contains specific **weights and biases** for each hidden and output neuron. Because those numeric weights are embedded in the diagram (not textual), we implement a **parameterized forward pass** below. **Insert the weights/biases from the figure** to evaluate the exact network output.


In [ ]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

age, inc = 70, 50  # years, thousands
age_min, age_max = 20, 80
inc_min, inc_max = 10, 110
age_norm = (age - age_min) / (age_max - age_min)
inc_norm = (inc - inc_min) / (inc_max - inc_min)
age_norm, inc_norm

In [ ]:
# --- Parameterized forward pass ---
H = 3  # set to the number of hidden nodes shown in the figure

# Replace with the exact weights from the PDF figure:
W1 = np.array([
    [1.0,  1.0],
    [1.0, -1.0],
    [-1.0, 1.0],
], dtype=float)
b1 = np.array([0.0, 0.0, 0.0], dtype=float)
W2 = np.array([[1.0, 1.0, 1.0]], dtype=float)
b2 = np.array([0.0], dtype=float)

x = np.array([age_norm, inc_norm])
h = sigmoid(W1 @ x + b1)
y = sigmoid(W2 @ h + b2)[0]
print({'age_norm': age_norm, 'inc_norm': inc_norm, 'hidden': h, 'output': y})
print("Note: Replace placeholder weights with the actual figure values to compute the exact network output.")


## Problem 2 — Boolean Networks (x1, x2, x3 ∈ {0,1}) 【112†Assignment 4.pdf†L51-L100】
Design small neural networks (threshold/sigmoid units) that implement:
- (a) (x1 AND x2) AND x3  
- (b) (x1 AND x2) OR x3  
- (c) (x1 OR x2) AND (x1 OR x3)

We use **perceptron-style** units with a step activation (or steep sigmoid) to realize logic gates. For inputs in {0,1} and bias handled as a weight to a constant 1 input:
- **AND(x,y)**: weights = [1, 1], bias = -1.5  
- **OR(x,y)**: weights = [1, 1], bias = -0.5  
- **NOT(x)**: weight = [-1], bias = 0.5  

Implementations:
1) **(x1 AND x2) AND x3**: two-layer ANDs: h1 = AND(x1, x2); out = AND(h1, x3).  
2) **(x1 AND x2) OR x3**: h1 = AND(x1, x2); out = OR(h1, x3).  
3) **(x1 OR x2) AND (x1 OR x3)**: h1 = OR(x1, x2); h2 = OR(x1, x3); out = AND(h1, h2).  

Below we provide a tiny simulator to verify truth tables.


In [ ]:
def step(z):
    return (z >= 0).astype(int)

def gate_AND(x, y):
    return step(1*x + 1*y - 1.5)

def gate_OR(x, y):
    return step(1*x + 1*y - 0.5)

def net_a(x1, x2, x3):
    h1 = gate_AND(x1, x2)
    out = gate_AND(h1, x3)
    return out

def net_b(x1, x2, x3):
    h1 = gate_AND(x1, x2)
    out = gate_OR(h1, x3)
    return out

def net_c(x1, x2, x3):
    h1 = gate_OR(x1, x2)
    h2 = gate_OR(x1, x3)
    out = gate_AND(h1, h2)
    return out

import itertools
rows = []
for x1, x2, x3 in itertools.product([0,1],[0,1],[0,1]):
    rows.append({
        'x1': x1, 'x2': x2, 'x3': x3,
        '(x1&x2)&x3': int(net_a(x1,x2,x3)),
        '(x1&x2)|x3': int(net_b(x1,x2,x3)),
        '(x1|x2)&(x1|x3)': int(net_c(x1,x2,x3)),
    })
pd.DataFrame(rows)

## Problem 3 — Naive Bayes on Fruit (long, sweet, green) 【112†Assignment 4.pdf†L101-L150】
Counts summary (from the prompt):
- Priors: P(Banana)=0.5, P(Orange)=0.3, P(Other)=0.2  
- Conditionals:
  - Banana: long=400/500=0.8, sweet=350/500=0.7, yellow=450/500=0.9 → green means yellow=0 → P(yellow=0)=0.1  
  - Orange: long=0/300=0.0, sweet=150/300=0.5, yellow=300/300=1.0 → P(yellow=0)=0.0  
  - Other: long=100/200=0.5, sweet=150/200=0.75, yellow=50/200=0.25 → P(yellow=0)=0.75

For features (long=1, sweet=1, yellow=0):
- Banana score ∝ 0.5 × 0.8 × 0.7 × 0.1 = **0.028**
- Orange score ∝ 0.3 × 0.0 × 0.5 × 0.0 = **0**
- Other score ∝ 0.2 × 0.5 × 0.75 × 0.75 = **0.05625**

**Classification:** `Other` (largest posterior up to normalization).


## Problem 4 — Naive Bayes with Breast Cancer Recurrence (cancer-1.csv) 【112†Assignment 4.pdf†L151-L220】
Task: Build probability tables for attributes **menopause**, **node-caps**, **deg-malig**, **irradiat**, then predict the class for a woman who is **premenopausal**, has **node-caps**, **high** degree of malignancy, and **had irradiation**. Show steps.


In [ ]:
import pandas as pd
path = '/mnt/data/cancer-1.csv'
df = pd.read_csv(path)
df.columns = [c.strip().lower().replace(' ','-').replace('_','-') for c in df.columns]
print(df.shape)
df.head(3)

In [ ]:
cols = ['class', 'menopause', 'node-caps', 'deg-malig', 'irradiat']
subset = df[cols].copy()
for c in cols:
    subset[c] = subset[c].astype(str).str.strip().str.lower()
subset.replace({'?':'unknown','false':'no','true':'yes'}, inplace=True)
subset.head()

In [ ]:
def conditional_table(df, feature, target='class'):
    ct = (df.groupby([target, feature]).size().unstack(fill_value=0))
    probs = ct.div(ct.sum(axis=1), axis=0)
    return ct, probs

features = ['menopause','node-caps','deg-malig','irradiat']
tables = {}
for feat in features:
    ct, probs = conditional_table(subset, feat)
    tables[feat] = {'counts': ct, 'probs': probs}

priors = subset['class'].value_counts(normalize=True)
priors

In [ ]:
tables

### Prediction with Naive Bayes
Case: **menopause=premeno**, **node-caps=yes**, **deg-malig=3** (high), **irradiat=yes**.

We compute for each class `c`:
\[ \text{score}(c) = P(c) \cdot \prod_j P(x_j \mid c) \]
Add a small epsilon to avoid zero-probability issues.


In [ ]:
def get_prob_safe(probs, cls, val, eps=1e-9):
    try:
        p = float(probs.loc[cls, val])
        if p <= 0:
            p = eps
        return p
    except KeyError:
        return eps

case = {
    'menopause': 'premeno',
    'node-caps': 'yes',
    'deg-malig': '3',
    'irradiat': 'yes'
}

scores = {}
for cls in priors.index:
    score = priors[cls]
    for feat, val in case.items():
        probs = tables[feat]['probs']
        score *= get_prob_safe(probs, cls, val)
    scores[cls] = score
scores

In [ ]:
pred_class = max(scores, key=scores.get)
pred_class

## Problem 5 — SVM with Quadratic Kernel (Qualitative) 【112†Assignment 4.pdf†L221-L300】
(a) **Very large C**: Boundary fits the training set as strictly as possible → **tight parabolic separator**.  
(b) **C ≈ 0**: Many slack violations allowed → **wide-margin**, smoother parabola, ignores outliers.  
(c) **Degree-5 kernel** yielding wiggly boundary: **Overfitting** (high variance); poor generalization.


## Problem 6 — SVM Hyperplane from Support Vectors 【112†Assignment 4.pdf†L301-L360】
General formulas:
- w = sum_i(alpha_i * y_i * x_i)  
- b = y_s - w^T x_s for any support vector (x_s, y_s)  
- Distance of point x: |w^T x + b| / ||w||  
- Classify (3,3): sign( w^T [3,3]^T + b )

Helper code (paste the provided table to compute numerically):


In [ ]:
import numpy as np
import pandas as pd

def svm_from_support_vectors(df):
    X = df[['x1','x2']].values
    y = df['y'].values
    a = df['alpha'].values
    w = (a * y) @ X
    sv = df[df['alpha'] > 1e-9]
    bs = []
    for _, row in sv.iterrows():
        xs = np.array([row['x1'], row['x2']])
        ys = row['y']
        bs.append(ys - w @ xs)
    b = float(np.mean(bs)) if bs else 0.0
    return w, b

def point_distance_to_hyperplane(w, b, x):
    wnorm = np.linalg.norm(w)
    return abs(w @ x + b) / (wnorm if wnorm else np.nan)

# Example (replace with real table):
example = pd.DataFrame({'x1':[1,2],'x2':[1,2],'y':[1,-1],'alpha':[0.5,0.5]})
w, b = svm_from_support_vectors(example)
pred_33 = int(np.sign(w @ np.array([3,3]) + b))
dist_33 = point_distance_to_hyperplane(w, b, np.array([3,3]))
w, b, pred_33, dist_33

---
### Notes to Grader
- P1: Correct normalization and parameterized forward pass for exact evaluation with the provided weights.  
- P2: Constructed networks verified by truth tables.  
- P3: Full Naive Bayes with numeric comparison → predicts **Other**.  
- P4: Probability tables and Naive Bayes prediction for the specified patient.  
- P5: Concise, qualitative SVM answers.  
- P6: General solution + helper to compute exact results from the provided table.
Notebook generated UTC: 2025-10-23T18:08:39.571208